# eqc-models Brief


## Modeling-first Pattern

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qci-wdyk/eqc-models-tutorial/blob/main/tutorial02-eqc-models.ipynb)

<img style="display: block; margin-left: auto; margin-right: auto;" src="images/model-operator-solver.png" />

**Solvers call for specific operators.**

<img style="display: block; margin-left: auto; margin-right: auto;" src="images/operator-solver.png" />

## Important Base Classes from eqc-models

- `EQCModel`
- `QuadraticModel` - offers both QUBO and Polynomial operators
- `PolynomialModel` - offers both QUBO and Polynomial operators
- `ConstraintsMixIn`
- `InequalityConstraintMixin`


## Intended Usage

`EQCModel` is an "abstract" class which defines some key interfaces. 
`QuadraticModel` and `PolynomialModel` are useful on their own, but the can also be used to build much more useful objects. 

Uses the **mixin design pattern** to incorporate functionality that can be shared across models and solvers.


### `QuadraticModel`

$$
E(x)=\sum_i C_ix_i + \sum_{ij} J_{ij}x_ix_j
$$

Specify a vector of linear coefficients and a matrix of quadratic coefficients. We'll call these `C` and `J`.


In [ ]:
import sys

# Check if we are in Google Colab
if 'google.colab' in sys.modules:
  print("Running in Google Colab. Performing setup...")
  # Installation process for EQC-models to be operable in Google Colab
  get_ipython().run_line_magic('shell', 'sudo apt-get update -y')
  get_ipython().run_line_magic('shell', 'sudo apt-get install software-properties-common')
  get_ipython().run_line_magic('shell', 'sudo add-apt-repository ppa:deadsnakes/ppa -y')
  get_ipython().run_line_magic('shell', 'sudo apt-get update -y')
  get_ipython().run_line_magic('shell', 'sudo apt-get install python3.10 python3.10-distutils -y')
  get_ipython().run_line_magic('shell', 'sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1')
  get_ipython().run_line_magic('shell', 'wget https://bootstrap.pypa.io/get-pip.py')
  get_ipython().run_line_magic('shell', 'python3.10 get-pip.py')
  get_ipython().run_line_magic('shell', 'python3.10 -m pip install eqc_models numpy matplotlib')
  print("Colab setup complete.")

else:
  print("Running in a local environment. Assuming packages are installed.")

# Now, your regular code can follow.
# In Colab, you still need to use the !python3.10 trick to run your scripts.
# In your local notebook, you can just run code in the cells as usual.

In [1]:
import numpy as np
from eqc_models.base import QuadraticModel

C = np.array([-1.0, -1.0, -1.0])
J = np.array([[0.0, 1.0, 1.0],
              [0.0, 0.0, 1.0],
              [0.0, 0.0, 0.0]])
model = QuadraticModel(C, J)
model.upper_bound = np.ones((3, ))
model.qubo.Q

array([[-1. ,  0.5,  0.5],
       [ 0.5, -1. ,  0.5],
       [ 0.5,  0.5, -1. ]])

In [2]:
model.upper_bound = 3*np.ones((3,))
model.qubo.Q

array([[-1. ,  0. ,  0.5,  1. ,  0.5,  1. ],
       [ 0. , -2. ,  1. ,  2. ,  1. ,  2. ],
       [ 0.5,  1. , -1. ,  0. ,  0.5,  1. ],
       [ 1. ,  2. ,  0. , -2. ,  1. ,  2. ],
       [ 0.5,  1. ,  0.5,  1. , -1. ,  0. ],
       [ 1. ,  2. ,  1. ,  2. ,  0. , -2. ]], dtype=float32)

In [3]:
from eqc_models.solvers import Dirac1CloudSolver
solver = Dirac1CloudSolver()
Q = model.qubo.Q
# qn = Q.shape[0]
# qubomodel = QuadraticModel(np.zeros((qn,)), Q)
# qubomodel.upper_bound = np.ones((qn,))
response = solver.solve(model) # qubomodel)
# response["results"]["solutions"], model.qubo.evaluate(np.array(response["results"]["solutions"][0]))
solution = model.decode(np.array(response["results"]["solutions"][0]), "qubo")
solution, model.evaluate(solution)

2025-03-14 23:01:17 - Dirac allocation balance = 0 s (unmetered)
2025-03-14 23:01:17 - Job submitted: job_id='67d5099d00e804f113aefca4'
2025-03-14 23:01:17 - QUEUED
2025-03-14 23:01:20 - RUNNING
2025-03-14 23:01:30 - COMPLETED
2025-03-14 23:01:33 - Dirac allocation balance = 0 s (unmetered)


(array([0, 0, 3], dtype=int64), -3.0)

### `PolynomialModel`

$$
E(x)=\sum_i C_i x_i + \sum_i\sum_j J_{ij}x_ix_j + \sum_i\sum_j\sum_k T_{ijk} x_ix_jx_k + \sum_i\sum_j\sum_k\sum_l Q_{ijkl} x_ix_jx_kx_l+ \sum_i\sum_j\sum_k\sum_l\sum_m P_{ijklm} x_ix_jx_kx_lx_m .
$$

### `ConstraintsMixIn`

The constraint mixin defines a standard method to convert a linear system of equality constraints into a penalty function, which is 0 for feasible solutions and positive for infeasible solutions.
$$
Ax=b\Rightarrow P(x)=(Ax-b)^2
$$

### `InequalityConstraintMixIn`

This adds a `senses` attribute which allows the definition of constraints with inequalities. These become equality constraints and are converted to penalties using the `Constraints


In [4]:
from eqc_models.base.constraints import ConstraintsMixIn
from eqc_models.base.polynomial import PolynomialModel
class ConstraintExample(ConstraintsMixIn, PolynomialModel):
    def __init__(self, coefficients, indices, lhs, rhs):
        self.constraints = lhs, rhs
        super(ConstraintExample,self).__init__(coefficients, indices)

In [7]:
A = np.array([[1, 0, -1]])
b = np.array([0])
coeff = model.polynomial.coefficients
indices = model.polynomial.indices
constraint_model = ConstraintExample(coeff, indices, A, b)
constraint_model.upper_bound = np.ones((3,))
constraint_model.penalty_multiplier = alpha = 2
response = solver.solve(constraint_model)
solution = np.array(response["results"]["solutions"][0])
np.array(solution), constraint_model.polynomial.pure_evaluate(solution) + alpha * constraint_model.offset

2025-03-14 23:04:11 - Dirac allocation balance = 0 s (unmetered)
2025-03-14 23:04:11 - Job submitted: job_id='67d50a4b00e804f113aefcaa'
2025-03-14 23:04:11 - QUEUED
2025-03-14 23:04:14 - RUNNING
2025-03-14 23:04:24 - COMPLETED
2025-03-14 23:04:27 - Dirac allocation balance = 0 s (unmetered)


(array([1, 0, 1]), array([-1.]))

## Problem Type Classes

Two categories- ML and Decision Optimization.

### Decision Optimization

- `QAPModel`
- `SetPartitionModel`
- `SetCoverModel`
- `MTZTSPModel`
- `AllocationModel`
- `PortMomentum`
- `MaxCutModel`
- `GraphPartitionModel`

### Machine Learning

- `QBoostClassifier`
- `QSVMClassifier`
- `PCA`
- `LinearRegression`

### Algorithms

- `PenaltyMultiplierAlgorithm`

In [ ]:
# Download and execute set up script
!wget -O py310.sh https://raw.githubusercontent.com/j3soon/colab-python-version/main/scripts/py310.sh
!bash py310.sh